# Recap of Large Language Models

Learn the basics

## Recap LLMs ~ Generate an answer to a single question

In [ ]:
from dotenv import load_dotenv
import logging

from bring_a_crew.setup_logging import setup_logging
from ollama import chat, generate


In [ ]:
MODEL = "phi4"

_ = load_dotenv()
setup_logging()
main_log = logging.getLogger("main")
main_log.setLevel(logging.INFO)


In [ ]:
def run_question(question):
    llm_response = generate(
        prompt=question,
        model=MODEL,
        options={
            "temperature": 0.5
        }
    )
    main_log.debug(llm_response)
    return llm_response.response


In [ ]:
print(run_question("When is Jettro available?"))

## Be more specific
Ask a better question

In [ ]:
print(run_question("I need to make an appointment with Jettro. When is he available?"))

## Provide a system prompt with some guidelines

With a system message, you tell the LLM more about the role

In [ ]:
def run_question_with_system_prompt(question):
    llm_response = generate(
        prompt=question,
        model=MODEL,
        options={
            "temperature": 0.5,
        },
        system=(
            "You are a scheduling assistant. You help in checking the availability of people."
            "Do not make up availability if you do not know the person's schedule."
            "Answer in short sentences, stick to the answer to the question."
        )
    )
    main_log.debug(llm_response)
    return llm_response.response


In [ ]:
print(run_question_with_system_prompt("When is Jettro available?"))

## Provide a context with the required information
The way you obtain the context is not important for the LLM.

In [ ]:
context = (
    "This is the agenda of people in our office:\n"
    "- Jettro is available on Monday and Thursday;\n"
    "- Joey is available on Tuesday, Thursday, and Friday;\n"
    "- Daniel is available from Monday to Thursday."
)

print(run_question_with_system_prompt(f"{context}\nWhen is Jettro available?"))

In [ ]:
print(run_question_with_system_prompt(f"When is Jettro, Joey and Daniel available together?"))

## Give the LLM Memory
By keeping the messages in a memory, the LLM can keep using the context. Note that we switched from the `generate` function to the `chat` function.

In [ ]:
def run_chat(question, messages=None):
    _messages = [
        {"role": "system", "content": (
            "You are a scheduling assistant. You help in checking the availability of people."
            "Do not make up availability if you do not know the person's schedule."
            "Answer in short sentences, stick to the answer to the question."
        )},
    ]
    if messages is not None:
        _messages.extend(messages)

    _messages.append({"role": "user", "content": question})

    llm_response = chat(
        model=MODEL,
        options={
            "temperature": 0.5
        },
        messages=_messages
    )
    main_log.debug(llm_response)
    _messages.append({"role": "assistant", "content": llm_response.message.content})
    return llm_response.message.content, _messages[1:]


response_messages = None

In [ ]:
response, response_messages = run_chat(question=f"{context}\nWhen is Jettro available?", messages=response_messages)
print(response)

In [ ]:
response, response_messages = run_chat(question=f"When are Jettro and Joey both available?", messages=response_messages)
print(response)

## Give the LLM a Tool to ask schedule information
By using a Tool, we can have a dynamic context. The Tool is a function that the LLM knows how to call.

In [ ]:
# This is the tool
def find_person_availability(name: str) -> str:
    main_log.info(f"Finding availability for {name}")
    if name.lower() == "jettro":
        return "Jettro is available on Tuesday and Thursday."
    elif name.lower() == "daniel":
        return "Daniel is available on Monday to Thursday."
    elif name.lower() == "joey":
        return "Joey is available on Thursday and Friday."
    else:
        return "I do not know the availability of this person."


In [ ]:
MODEL = "llama3.2"

def run_chat_with_tool(question, messages=None):
    _messages = [
        {
            "role": "system", "content": (
            "You are a scheduling assistant. You help in checking the availability of people."
            "Do not make up availability if you do not know the person's schedule."
            "Answer in short sentences, stick to the answer to the question."
        )}]
    if messages is not None:
        _messages.extend(messages)

    if question is not None:
        _messages.append({"role": "user", "content": question})

    llm_response = chat(
        model=MODEL,
        options={
            "temperature": 0
        },
        messages=_messages,
        tools=[find_person_availability]
    )
    main_log.debug(llm_response)

    if llm_response.message.tool_calls:
        calls = []
        for tool_call in llm_response.message.tool_calls:
            function_name = tool_call.function.name
            argument = tool_call.function.arguments["name"]
            calls.append((function_name, argument))
        return calls, _messages

    _messages.append({"role": "assistant", "content": llm_response.message.content})
    return llm_response.message.content, _messages[1:]

response_messages = None

In [ ]:
response, response_messages = run_chat_with_tool(question=f"When are Jettro and Joey both available?", messages=response_messages)
print(response)

In [ ]:
# Call the tool
for call in response:
    availability = find_person_availability(call[1])
    response_messages.append({"role": "tool", "content": availability, "name": "find_person_availability"})

response, response_messages = run_chat_with_tool(question=None, messages=response_messages)
print(response)
